In [0]:
# Source with default settings
%pip install azure-eventhub
from azure.eventhub import EventHubProducerClient, EventHubConsumerClient, EventData

import pyspark.pandas as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *



from pyspark import SparkContext
sc =SparkContext.getOrCreate()

connectionString = "Endpoint=sb://streamdata-f9019e-ns.servicebus.windows.net/;SharedAccessKeyName=rule;SharedAccessKey=Zebuc91qznwl91w9y3Ec1ZEM0YXZXlFX9+AEhHtUxe8=;EntityPath=streamdata-f9019e-ns"
ehConf = {}
ehConf['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)

df = spark \
  .readStream \
  .format("eventhubs") \
  .options(**ehConf) \
  .load()

sales_orders_df = df.select("body").withColumn("body", col("body").cast("string"))

sales_orders_df.writeStream.format("delta")\
   .outputMode("append")\
   .option("checkpointLocation", "/tmp/delta/events/_checkpoints/")\
   .toTable("sales_order_raw_stream")



Python interpreter will be restarted.
  Using cached azure_eventhub-5.11.2-py3-none-any.whl (305 kB)
  Using cached azure_core-1.27.1-py3-none-any.whl (174 kB)
  Using cached typing_extensions-4.6.3-py3-none-any.whl (31 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-0b7057c5-cffd-43ea-9f05-5dc88ebfb911
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
Python interpreter will be restarted.
Out[1]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f21707cdfd0>

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

sales_orders_schema = StructType(
    [
        StructField(
            "clicked_items", ArrayType(ArrayType(StringType(), True), True), True
        ),
        StructField("customer_id", LongType(), True),
        StructField("customer_name", StringType(), True),
        StructField("number_of_line_items", LongType(), True),
        StructField("order_datetime", StringType(), True),
        StructField("order_number", LongType(), True),
        StructField(
            "ordered_products",
            ArrayType(
                StructType(
                    [
                        StructField("id", StringType(), True),
                        StructField("qty", IntegerType(), True),
                        StructField("curr", StringType(), True),
                        StructField("name", StringType(), True),
                        StructField("unit", StringType(), True),
                        StructField("price", IntegerType(), True),
                        StructField(
                            "promotion_info",
                            StructType(
                                [
                                    StructField("promo_id", IntegerType(), True),
                                    StructField("promo_qty", IntegerType(), True),
                                    StructField("promo_disc", DecimalType(3, 2), True),
                                    StructField("promo_item", StringType(), True),
                                ]
                            ),
                            True,
                        ),
                    ]
                ),
                True,
            ),
            True,
        ),
        StructField(
            "promo_info",
            ArrayType(
                StructType(
                    [
                        StructField("promo_id", IntegerType(), True),
                        StructField("promo_qty", IntegerType(), True),
                        StructField("promo_disc", DecimalType(3, 2), True),
                        StructField("promo_item", StringType(), True),
                    ]
                ),
                True,
            ),
            True,
        ),
    ]
  )

df1 = (spark.readStream.format("delta").table("sales_order_raw_stream")
       .select("*")
       .withColumn("body", regexp_replace("body", '"\\[', "["))
       .withColumn("body", regexp_replace("body", '\\]"', "]"))
       .withColumn("body", regexp_replace("body","\\\\", ""))
       .withColumn("body", expr("substring(body, 2, length(body))"))
       .withColumn("body", expr("substring(body, 1, length(body)-1)"))
       .select(from_json(col("body"), sales_orders_schema).alias("row"))
       .select("row.*")
       .withColumn("ordered_products", explode("ordered_products"))
       .withColumn("order_datetime", from_unixtime("order_datetime"))
       .withColumn("product_id", col("ordered_products").id)
       .withColumn("unit_price", col("ordered_products").price)
       .withColumn("quantity", col("ordered_products").qty)
       
      )
df1.writeStream.format("delta")\
   .outputMode("append")\
   .option("checkpointLocation", "/tmp/delta/eventschkforsilver1/_checkpoints/")\
   .toTable("sales_orders_cleansed")

Out[2]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f60c0f39ca0>

In [0]:
s = spark.readStream.format("delta").table("sales_orders_cleansed").alias("s")
p = spark.read.format("delta").table("dim_products").alias("p")
c = spark.read.format("delta").table("dim_customers").alias("c")

df2 = s.join(p, s.product_id == p.product_id, "inner").join(c, s.customer_id == c.customer_id, "inner").select("s.order_number","c.customer_key","p.product_key",col("s.order_datetime").cast("date").alias("order_date"),"s.unit_price","s.quantity",expr("s.unit_price * s.quantity").alias("total_price"),)

df2.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "/tmp/delta/eventscheckforgoldretailorg1/_checkpoints/")\
    .toTable("fact_sales_orders1")

Out[4]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f60c0f39df0>

In [0]:
s = spark.readStream.format("delta").table("sales_orders_cleansed").alias("s")
p = spark.read.format("delta").table("dim_products").alias("p")
c = spark.read.format("delta").table("dim_customers").alias("c")

df3 = s.join(p, s.product_id == p.product_id, "inner").join(c, s.customer_id == c.customer_id, "inner").groupBy("c.customer_key", "p.product_key").agg(sum("quantity").alias("total_quantity"),sum(expr("s.unit_price * s.quantity")).alias("sale_amount"),)

df3.writeStream.format("delta")\
    .outputMode("complete")\
    .option("checkpointLocation", "/tmp/delta/eventscheckforgoldcompleteretailorg1/_checkpoints/")\
    .toTable("fact_customer_sales")

Out[5]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f60c0143f70>